# [Tutorial] Web Search Agent with LangChain-Nimble

> **📌 Databricks Notebook** - This notebook is designed for Databricks environments. Import it into your Databricks workspace to run. While the code can be adapted for other environments, it's optimized for Databricks with built-in workspace authentication.

This notebook demonstrates how to build an intelligent web search agent using **langchain-nimble**. The agent can search the web in real-time and extract content from specific URLs, making it perfect for applications that need up-to-date information.

**What you'll learn:**
- Setting up the Nimble Search and Extract tools
- Creating an agent with web search capabilities
- Running queries with streaming responses
- Seeing real-time AI-powered web research in action

### Install Dependencies

First, install the required packages. This example uses Databricks for the LLM, but you can use any LangChain-compatible model (OpenAI, Anthropic, etc.).

In [ ]:
%pip install langchain langchain-nimble databricks-langchain python-dotenv langchain-openai
%restart_python

### Configure API Keys

After the kernel restarts, you'll need to set up your API keys:

1. **Nimble API Key** - Get your key at [nimbleway.com](https://app.nimbleway.com/account_settings/api_keys)
   - Set it in the server environment as `NIMBLE_API_KEY`
   - Or enter it when prompted by the code below

2. **Databricks Authentication**
   - Automatically configured using your workspace credentials

In [ ]:
import getpass
import os
import time

from databricks.sdk import WorkspaceClient

# Prompt the user to securely input the API key if not already set in the environment
if not os.environ.get("NIMBLE_API_KEY"):
    os.environ["NIMBLE_API_KEY"] = getpass.getpass("NIMBLE_API_KEY:\n")

# Fetch Databricks credentials from environment
w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value    

### Create the Web Search Agent

Now we'll initialize the agent with two powerful tools:

- **NimbleSearchTool** - Searches the web for relevant information
- **NimbleExtractTool** - Extracts full content from specific URLs

The agent will automatically decide when to use each tool based on your query.

**Note:** The example below uses `databricks-meta-llama-3-3-70b-instruct` as the model endpoint. Replace this with your own Databricks model serving endpoint name, or use a different LLM provider (OpenAI, Anthropic, etc.).

In [ ]:
from databricks_langchain import ChatDatabricks
from langchain.agents import create_agent

from langchain_nimble import NimbleExtractTool, NimbleSearchTool

search_tool = NimbleSearchTool()
extract_tool = NimbleExtractTool()

llm_model = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

agent = create_agent(
    model=llm_model,
    tools=[search_tool, extract_tool],
    system_prompt="You are a helpful assistant with access to real-time web information"
)

## Running the Agent

Let's ask the agent to find information from the web. The agent will:
1. Search the web using NimbleSearchTool
2. Extract relevant content if needed
3. Synthesize the information into a clear answer

We're using streaming mode to see the agent's thought process in real-time.

In [ ]:
# Example 1: Recent news search
query = "What are the latest developments in AI this week?"

# OR Example 2: Technical documentation
# query = "Find the Python FastAPI documentation and summarize how to create a REST API"

# OR Example 3: Company research
# query = "Find information about LangChain and what they do"

# OR Example 4: Comparative analysis
# query = "Compare the features of React and Vue.js frameworks"

# Start timing
start_time = time.time()

# Stream the agent's response in real-time
async for step in agent.astream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

# Print total execution time
elapsed_time = time.time() - start_time
print(f"\n\n{'=' * 80}")
print(f"Total execution time: {elapsed_time:.2f} seconds")
print(f"{'=' * 80}")

## Next Steps

**Explore More Features:**
- Use `deep_search=True` for comprehensive research with full page content
- Filter by domains with `include_domains` and `exclude_domains`
- Search recent news with `topic="news"` and date filters
- Extract content from multiple URLs at once

**Learn More:**
- [Full Documentation](https://github.com/Nimbleway/langchain-nimble)
- [API Reference](https://docs.nimbleway.com/)
- [More Examples](https://github.com/Nimbleway/langchain-nimble/tree/main/examples)

**Get Help:**
- [GitHub Issues](https://github.com/Nimbleway/langchain-nimble/issues)
- [Nimble Website](https://nimbleway.com/)